In [18]:
import pickle
import neat

from carl_genome import CarlLanderGenome
from carl_population import CarlPopulation
from carl_species import CarlSpecies, CarlSpeciesSet
from carl_reproduction import CarlReproduction
import matplotlib.pyplot as plt
import numpy as np
import re

In [7]:
class LanderGenome(neat.DefaultGenome):
    def __init__(self, key):
        super().__init__(key)
        self.discount = None

    def configure_new(self, config):
        super().configure_new(config)
        self.discount = 0.01 + 0.98 * random.random()

    def configure_crossover(self, genome1, genome2, config):
        super().configure_crossover(genome1, genome2, config)
        self.discount = random.choice((genome1.discount, genome2.discount))

    def mutate(self, config):
        super().mutate(config)
        self.discount += random.gauss(0.0, 0.05)
        self.discount = max(0.01, min(0.99, self.discount))

    def distance(self, other, config):
        dist = super().distance(other, config)
        disc_diff = abs(self.discount - other.discount)
        return dist + disc_diff

    def __str__(self):
        return "Reward discount: {0}\n{1}".format(self.discount,
                                                  super().__str__())


In [19]:
stat_pneat = pickle.load(open("paper_result/save_stats.p","rb"))
stat_neat = pickle.load(open("../../../neat-python-origin/examples/openai-lander/paper_result/save_stats2.p","rb"))

In [24]:
def plot_stats(stat1, stat2):
    """ Plots the population's average and best fitness. """
    if plt is None:
        warnings.warn("This display is not available due to a missing optional dependency (matplotlib)")
        return

    generation = range(len(stat1.most_fit_genomes))
    best_fitness = [c.fitness for c in stat1.most_fit_genomes]
    avg_fitness = np.array(stat1.get_fitness_mean())

    plt.plot(generation, avg_fitness, 'b-', label="PedigreeNEAT average", linewidth = 1)
    #plt.plot(generation, avg_fitness - stdev_fitness, 'g-.', label="-1 sd")
    plt.plot(generation, best_fitness, 'b--', label="PedigreeNEAT best", linewidth = 1)
    
    generation = range(len(stat2.most_fit_genomes))
    generation = range(1000)
    best_fitness = [c.fitness for c in stat2.most_fit_genomes]
    best_fitness = best_fitness[:1000]
    avg_fitness = np.array(stat2.get_fitness_mean())
    avg_fitness = avg_fitness[:1000]

    plt.plot(generation, avg_fitness, 'r-', label="NEAT average", linewidth = 1)
    #plt.plot(generation, avg_fitness - stdev_fitness, 'g-.', label="-1 sd")
    plt.plot(generation, best_fitness, 'r--', label="NEAT best", linewidth = 1)

    plt.title("Population's average and best fitness")
    plt.xlabel("Generations")
    plt.ylabel("Fitness")
    plt.grid()
    plt.legend(loc="best")

    plt.savefig("Fitness", format="eps", dpi=300)

    plt.close()

In [25]:
plot_stats(stat_pneat, stat_neat)

In [29]:
# time plot bar
result1 = "/home/jinwei/Documents/Git/neat-python-origin/examples/openai-lander/time.txt"
result2 = "/home/jinwei/Documents/Git/neat-python/examples/openai-lander/paper_result/result.txt"
time1 = []
with open(result1, "r") as f:
    lines = f.readlines()
    for line in lines:
        if ("population 5 generations time" in line):
            time = float(line.split(" ")[-1])
            time1.append(time)
time2 = []
with open(result2, "r") as f:
    lines = f.readlines()
    for line in lines:
        if ("population 5 generations time" in line):
            time = float(line.split(" ")[-1])
            time2.append(time)

In [37]:
time1 = time1[1:5]
time2 = time2[1:5]

In [63]:
time1 = [x/5.0 for x in time1]
time2 = [x/5.0 for x in time2]

In [96]:
def autolabel(rects):
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        height = rect.get_height()
        plt.text(rect.get_x() + rect.get_width()/2., 0.5+height,
                '%.2f' % height,
                ha='center', va='bottom')
        
def simulation_time(t1, t2):
    if plt is None:
        warnings.warn("This display is not available due to a missing optional dependency (matplotlib)")
        return
    t1 = np.array(t1)
    t2 = np.array(t2)
    avg_t1 = np.mean(t1)
    avg_t2 = np.mean(t2)
    std_t1 = np.std(t1)
    std_t2 = np.std(t2)
    print(std_t1, std_t2)
    b1 = plt.bar(1, avg_t1, 0.8, color='r', yerr=std_t1, label="NEAT")
    b2 = plt.bar(2.2, avg_t2, 0.8, color='b', yerr=std_t2, label="PedigreeNEAT")
    plt.xticks([1, 2.2],('NEAT', 'PedigreeNEAT'))
    plt.ylim(0,5)
    plt.ylabel("Simulation time for 1 generation")
    plt.xlabel("Genetic Algorithms")
    plt.legend(loc="best")
    #plt.title("Simulation Time of One Generation")
    autolabel(b1)
    autolabel(b2)
    plt.savefig("simulate.eps", format="eps", dpi=300)
    plt.close()

In [97]:
simulation_time(time1, time2)

0.36386272631424504 0.06545505781708247


In [100]:
# species change 
result1 = "/home/jinwei/Documents/Git/neat-python-origin/examples/openai-lander/paper_result/result2.txt"
result2 = "/home/jinwei/Documents/Git/neat-python/examples/openai-lander/paper_result/result.txt"
species1 = []
with open(result1, "r") as f:
    lines = f.readlines()
    for line in lines:
        if ("Population of" in line and "species:" in line):
            species1.append(int(line.split(" ")[-2]))
species2 = []
with open(result2, "r") as f:
    lines = f.readlines()
    for line in lines:
        if ("Population of" in line and "species:" in line):
            species2.append(int(line.split(" ")[-2]))

In [112]:
def species(s1, s2):
    if plt is None:
        warnings.warn("This display is not available due to a missing optional dependency (matplotlib)")
        return
    x1 = range(len(s1))
    x2 = range(len(s2))
    plt.plot(x1, s1, 'b-', label="PedigreeNEAT Species")
    plt.plot(x2, s2, 'r-', label="NEAT Species")

    plt.xlabel("Generations")
    plt.ylabel("Number of Species")
    plt.yticks(range(0,25,5))
    plt.grid()
    plt.legend(loc="best")

    plt.savefig("species", format="eps", dpi=300)

    plt.close()

In [113]:
species(species1, species2)

In [114]:
s1 = np.array(species1)
s2 = np.array(species2)
print(np.mean(s1), np.mean(s2), np.std(s1), np.std(s2))

12.072254335260116 13.402380952380952 2.601981778599028 4.510568969243665
